<a href="https://colab.research.google.com/github/helendang-glac/test-colab/blob/main/Copy_of_AI_Search_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/5ca3e740-cf30-11ef-a16d-9b898737f71b-Screen_Shot_2025_01_10_at_15.53.26.png" width=700 />

Tác giả: [ProtonX Team](https://protonx.coursemind.io/courses/677e0eeb02a8c600bdeb64e5/info)

Yêu cầu:
- [OpenAI key](https://platform.openai.com/settings/organization/api-keys) để làm backend LLMs
- [Serp API](https://serper.dev/api-key) key để tìm trên Google

In [ ]:
!pip install serpapi
!pip install google-search-results

In [ ]:
from openai import OpenAI
from google.colab import userdata

def reasoning_step(state, user_input, intermediate_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))

    # Construct the assistant message dynamically with the state, user input, and intermediate results
    messages = [
        {
            "role": "system",
            "content": (
                "You are a reasoning and acting agent. Based on the current state and user input, decide the next action.\n"
                f"State: {state}\n"
                f"User Input: {user_input}\n"
                f"Intermediate Results: {intermediate_results}\n\n"
                "Respond with one of these actions:\n"
                "- Search(query)\n"
                "- Do nothing\n"
                "- Summarize(results)\n"
            )
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "agent_schema",
                "schema": {
                    "type": "object",
                    "required": [],
                    "properties": {}
                },
                "strict": False
            }
        },
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "search_action",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": ["query"],
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query string to be submitted to the search engine."
                            }
                        },
                        "additionalProperties": False
                    },
                    "description": "Performs a search using the SERP API and returns the organic results."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "do_nothing",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": [],
                        "properties": {},
                        "additionalProperties": False
                    },
                    "description": "A function that performs no action."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "summarize_action",
                    "description": "Summarizes product results using OpenAI's GPT-4 model",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "results"
                    ],
                    "properties": {
                        "results": {
                        "type": "string",
                        "description": "The product results to summarize"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            }
        ],
        tool_choice="required",
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response

Giả sử chưa search bao giờ:

In [ ]:
assistant_message = reasoning_step(
    "start",
    "iphone 14 price?",
    []
).choices[0].message

In [ ]:
assistant_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iQc9jcrTRdiEHB5qYn4qUhtJ', function=Function(arguments='{"query":"iphone 14 price"}', name='search_action'), type='function')])

Đã search và có thông tin rồi

In [ ]:
assistant_message = reasoning_step(
    "start",
    "iphone 14 price?",
    [{
            "action": "search_action",
            "parameters": "iPhone 14 price",
            "result": "iPhone 15 is $1000"
    }
     ]
).choices[0].message

In [ ]:
assistant_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BtC2J5r5Wg0Z2RXGvPvhNQVE', function=Function(arguments='{"query":"iPhone 14 price"}', name='search_action'), type='function')])

Đánh giá chất lượng


```
Query: iphone 14 price?
Expect: trong output cuối cùng có giá $500

```




In [ ]:
def process_tool_calls(response):
    # Extract tool call information from the response
    tool_calls = response.tool_calls

    if tool_calls:
        # Extract tool function details
        tool_call_id = tool_calls[0].id
        tool_function_name = tool_calls[0].function.name
        tool_query_string = eval(tool_calls[0].function.arguments)

        # Print extracted details
        print('Tool Function Name:', tool_function_name)
        print('Tool Query String:', tool_query_string)

        return tool_function_name, tool_query_string

    else:
        print("No tool calls identified.")
        return None, None

process_tool_calls(assistant_message)

Tool Function Name: search_action
Tool Query String: {'query': 'iPhone 14 price'}


('search_action', {'query': 'iPhone 14 price'})

In [ ]:
def search_action(query):
    from serpapi import GoogleSearch
    api_key = userdata.get("serpapi")
    search = GoogleSearch({"q": query, "api_key": api_key, "num": 10})
    results = search.get_dict()

    info = []

    print('--->', results)

    for item in results['organic_results']:
        info.append(item['snippet'])
    return "\n".join(info)

In [ ]:
search_action("Iphone 14 price")

---> {'search_metadata': {'id': '6803107784f6c047a755e6bb', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/c405e49cd46d8d10/6803107784f6c047a755e6bb.json', 'created_at': '2025-04-19 02:54:47 UTC', 'processed_at': '2025-04-19 02:54:47 UTC', 'google_url': 'https://www.google.com/search?q=Iphone+14+price&oq=Iphone+14+price&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/c405e49cd46d8d10/6803107784f6c047a755e6bb.html', 'total_time_taken': 0.69}, 'search_parameters': {'engine': 'google', 'q': 'Iphone 14 price', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Iphone 14 price', 'total_results': 2020000000, 'time_taken_displayed': 0.31, 'organic_results_state': 'Results for exact spelling'}, 'product_result': {'title': 'Apple iPhone 14', 'reviews': 28000, 'rating': 4.6, 'pricing': [{'price': '$366.00', 'extracted_price': 366.0, 'link': 'https://www.walmart.com/ip/Apple-iPhone-14-2

'List Price: $449.00$449.00 Details. The List Price is the suggested retail price of a new product as provided by a manufacturer, supplier, or seller. Except ...\nBuy iPhone 14 ; From $599 ; or $24.95/mo. per month for 24 mo.monthsFootnote* ; Need a moment? Keep all your selections by saving this device to Your Saves, then ...\nIn April2025, iPhone 14 prices starts at $324.99, but prices vary based on storage, color, condition and carrier compatibility.\nThe ultimate iPhone ... From $999 or $41.62/mo. for 24 mo ... Learn more Buy ... Apple Intelligence.\nApple iPhone 14 · Special offers (4) · Get up to $800/line to break your contract. · Get 4 lines of unlimited for $25/mo. per line · Online only. Get $200 off a ...\nChoose an iPhone 14 model at Best Buy. Select an iPhone 14, iPhone 14 Plus, iPhone 14 Pro or iPhone 14 Pro Max model.\niPhone 14 prices start at $268 and cost $388 on average as of April 2025. iPhone 14 prices will continue to get cheaper over time. Apple phones hold ...\n

In [ ]:
def do_nothing():
    return None

def summarize_action(results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))
    prompt = f"Summarize the following product results:\n{results}"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

In [ ]:
import transformers
def summarize_using_open_source_model(doc):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tokenizer = AutoTokenizer.from_pretrained("minhtoan/t5-small-wikilingua_vietnamese")

    model = AutoModelForSeq2SeqLM.from_pretrained("minhtoan/t5-small-wikilingua_vietnamese")
    model.cuda()
    tokenized_text = tokenizer.encode(doc, return_tensors="pt").cuda()
    model.eval()
    summary_ids = model.generate(
                        tokenized_text,
                        max_length=256,
                        num_beams=5,
                        repetition_penalty=2.5,
                        length_penalty=1.0,
                        early_stopping=True
                    )
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [ ]:
def answer_question(question, search_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))
    prompt = f"Answer the following question using the provided search results:\n\nQuestion: {question}\n\nSearch Results:\n{search_results}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content


In [ ]:
function_mapping = {
    "search_action": search_action,
    "do_nothing": do_nothing,
    "summarize_action": summarize_using_open_source_model,
    "answer_question": answer_question,
}



In [ ]:
function_mapping["search_action"]("Iphone 14 price")

---> {'search_metadata': {'id': '6803107784f6c047a755e6bb', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/c405e49cd46d8d10/6803107784f6c047a755e6bb.json', 'created_at': '2025-04-19 02:54:47 UTC', 'processed_at': '2025-04-19 02:54:47 UTC', 'google_url': 'https://www.google.com/search?q=Iphone+14+price&oq=Iphone+14+price&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/c405e49cd46d8d10/6803107784f6c047a755e6bb.html', 'total_time_taken': 0.69}, 'search_parameters': {'engine': 'google', 'q': 'Iphone 14 price', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Iphone 14 price', 'total_results': 2020000000, 'time_taken_displayed': 0.31, 'organic_results_state': 'Results for exact spelling'}, 'product_result': {'title': 'Apple iPhone 14', 'reviews': 28000, 'rating': 4.6, 'pricing': [{'price': '$366.00', 'extracted_price': 366.0, 'link': 'https://www.walmart.com/ip/Apple-iPhone-14-2

'List Price: $449.00$449.00 Details. The List Price is the suggested retail price of a new product as provided by a manufacturer, supplier, or seller. Except ...\nBuy iPhone 14 ; From $599 ; or $24.95/mo. per month for 24 mo.monthsFootnote* ; Need a moment? Keep all your selections by saving this device to Your Saves, then ...\nIn April2025, iPhone 14 prices starts at $324.99, but prices vary based on storage, color, condition and carrier compatibility.\nThe ultimate iPhone ... From $999 or $41.62/mo. for 24 mo ... Learn more Buy ... Apple Intelligence.\nApple iPhone 14 · Special offers (4) · Get up to $800/line to break your contract. · Get 4 lines of unlimited for $25/mo. per line · Online only. Get $200 off a ...\nChoose an iPhone 14 model at Best Buy. Select an iPhone 14, iPhone 14 Plus, iPhone 14 Pro or iPhone 14 Pro Max model.\niPhone 14 prices start at $268 and cost $388 on average as of April 2025. iPhone 14 prices will continue to get cheaper over time. Apple phones hold ...\n

In [ ]:
# function_to_call = function_mapping[tool_function_name]
# function_response = function_to_call(**tool_query_string)

In [ ]:
from openai import OpenAI
from google.colab import userdata

def reasoning_step_advanced(state, user_input, intermediate_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))

    # Construct the assistant message dynamically with the state, user input, and intermediate results
    messages = [
        {
            "role": "system",
            "content": (
                "You are a reasoning and acting agent. Based on the current state and user input, decide the next action.\n"
                f"State: {state}\n"
                f"User Input: {user_input}\n"
                f"Intermediate Results: {intermediate_results}\n\n"
                "Respond with one of these actions:\n"
                "- Search(query)\n"
                "- Do nothing\n"
                "- answer_question(results)\n"
            )
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "agent_schema",
                "schema": {
                    "type": "object",
                    "required": [],
                    "properties": {}
                },
                "strict": False
            }
        },
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "search_action",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": ["query"],
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query string to be submitted to the search engine."
                            }
                        },
                        "additionalProperties": False
                    },
                    "description": "Performs a search using the SERP API and returns the organic results."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "do_nothing",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": [],
                        "properties": {},
                        "additionalProperties": False
                    },
                    "description": "A function that performs no action."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "summarize_action",
                    "description": "Summarizes product results using OpenAI's GPT-4 model",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "results"
                    ],
                    "properties": {
                        "results": {
                        "type": "string",
                        "description": "The product results to summarize"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "answer_question",
                    "description": "Answer question using the search result",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "question",
                        "search_results"
                    ],
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The question to be answered"
                        },
                        "search_results": {
                            "type": "string",
                            "description": "The search results from api to answer the question"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            }
        ],
        tool_choice="required",
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response

In [ ]:
def react_agent(user_input):
    state = {"status": "start", "history": []}  # Initialize the state with status and history
    intermediate_results = []

    while True:
        # Reasoning step: Decide the next action
        action_response = reasoning_step_advanced(state, user_input, intermediate_results).choices[0].message

        action_response = process_tool_calls(action_response)
        tool_function_name, tool_query_string = action_response

        if tool_function_name == "do_nothing":
            return intermediate_results[-1]["result"]
            break

        # Execute the selected action
        function_to_call = function_mapping[tool_function_name]
        action_result = function_to_call(**tool_query_string)

        print(action_result)

        # Save intermediate results
        intermediate_results.append({
            "action": tool_function_name,
            "parameters": tool_query_string,
            "result": action_result
        })

        # Update state
        state.update({
            "status": "in_progress",
            "last_action": tool_function_name,
            "last_parameters": tool_query_string
            })

    return intermediate_results

user_query = "Thời tiết hôm nay có phù hợp để đi bơi ở Hồ Gươm không?"
final_answer = react_agent(user_query)
print(f"Final Output: {final_answer}")

Tool Function Name: search_action
Tool Query String: {'query': 'thời tiết Hồ Gươm hôm nay'}
---> {'search_metadata': {'id': '6803069172aec5173086bafb', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/29c77579b88d3c58/6803069172aec5173086bafb.json', 'created_at': '2025-04-19 02:12:33 UTC', 'processed_at': '2025-04-19 02:12:33 UTC', 'google_url': 'https://www.google.com/search?q=th%E1%BB%9Di+ti%E1%BA%BFt+H%E1%BB%93+G%C6%B0%C6%A1m+h%C3%B4m+nay&oq=th%E1%BB%9Di+ti%E1%BA%BFt+H%E1%BB%93+G%C6%B0%C6%A1m+h%C3%B4m+nay&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/29c77579b88d3c58/6803069172aec5173086bafb.html', 'total_time_taken': 0.5}, 'search_parameters': {'engine': 'google', 'q': 'thời tiết Hồ Gươm hôm nay', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'thời tiết Hồ Gươm hôm nay', 'total_results': 2120000, 'time_taken_displayed': 0.26, 'organic_results_state': 'Results for exac